In [1]:
!pip install booknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.7 MB/s eta 0:00:00
  Created wheel for booknlp: filename=booknlp-1.0.7-py3-none-any.whl size=2420743 sha256=e22b2deaec4b56f97fd308c17754f9f4874138333c119051a9805b4e71a00b5c
  Stored in directory: /root/.cache/pip/wheels/bb/f5/b4/2ee6b317da6491ff01b742be603af81ceea156427f1f4be38a
Successfully built booknlp


In [ ]:
!unzip '/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip'

Archive:  /content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/scifiTVshows.zip
   creating: OriginalStoriesSeparated/
  inflating: OriginalStoriesSeparated/Babylon5.txt  
  inflating: OriginalStoriesSeparated/DoctorWhoSpinoffs.txt  
  inflating: OriginalStoriesSeparated/DoctorWho.txt  
  inflating: OriginalStoriesSeparated/Fringe.txt  
  inflating: OriginalStoriesSeparated/Farscape.txt  
  inflating: OriginalStoriesSeparated/Futurama.txt  
  inflating: OriginalStoriesSeparated/Stargate.txt  
  inflating: OriginalStoriesSeparated/StarTrek.txt  
  inflating: OriginalStoriesSeparated/StarWarsBooks.txt  
  inflating: OriginalStoriesSeparated/StarWarsRebels.txt  
  inflating: OriginalStoriesSeparated/XFiles.txt  
  inflating: scifi-test.txt          
  inflating: scifi-train.txt         
  inflating: scifi-val.txt           


In [2]:
from booknlp.booknlp import BookNLP

model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big"
	}
	
booknlp=BookNLP("en", model_params)

using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'big'}
downloading entities_google_bert_uncased_L-6_H-768_A-12-v1.0.model
downloading coref_google_bert_uncased_L-12_H-768_A-12-v1.0.model
downloading speaker_google_bert_uncased_L-12_H-768_A-12-v1.0.1.model


--- startup: 103.748 seconds ---


In [3]:
# Input file to process
input_file="/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/SampleBabylon.txt"

# Output directory to store resulting files in
output_directory="test_booknlp/"

# File within this directory will be named ${book_id}.entities, ${book_id}.tokens, etc.
book_id="sample_babylon"

booknlp.process(input_file, output_directory, book_id)

--- spacy: 1.486 seconds ---
--- entities: 29.542 seconds ---
--- quotes: 0.017 seconds ---
--- attribution: 2.125 seconds ---
--- name coref: 0.022 seconds ---
--- coref: 41.735 seconds ---
--- TOTAL (excl. startup): 74.993 seconds ---, 2951 words


In [ ]:
# import json 
# # reading the data from the file
# with open('/content/test_booknlp/sample_babylon.book') as f:
#     data = f.read()
  
# print("Data type before reconstruction : ", type(data))
      
# # reconstructing the data as a dictionary
# js = json.loads(data)

Data type before reconstruction :  <class 'str'>


In [ ]:
# print(js)

In [ ]:
# for i in js:
#   for j in js[i][0]:
#     print(j)
#     print(js[i][0][j])

In [ ]:
# print(js['characters'][0]['patient'])

In [4]:
# import json
# from collections import Counter

In [ ]:
# def proc(filename):
#     with open(filename) as file:
#         data=json.load(file)
#     return data

In [ ]:
# data=proc("/content/test_booknlp/sample_babylon.book")

In [ ]:
# def get_counter_from_dependency_list(dep_list):
#     counter=Counter()
#     for token in dep_list:
#         term=token["w"]
#         tokenGlobalIndex=token["i"]
#         counter[term]+=1
#     return counter

In [ ]:
# for character in data['characters']:
#   print(character)

In [ ]:
# for character in data['characters']:
#   # print(character)
#     agentList=character["agent"]
#     patientList=character["patient"]
#     possList=character["poss"]
#     modList=character["mod"]

#     character_id=character["id"]
#     count=character["count"]

#     referential_gender_distribution=referential_gender_prediction="unknown"
    
#     if character["g"] is not None and character["g"] != "unknown":
#         referential_gender_distribution=character["g"]["inference"]
#         referential_gender=character["g"]["argmax"]

#     mentions=character["mentions"]
#     proper_mentions=mentions["proper"]
#     max_proper_mention=""

#     # just print out information about named characters
#     if len(mentions["proper"]) > 0:
#         max_proper_mention=mentions["proper"][0]["n"]
        
#         print(character_id, count, max_proper_mention, referential_gender)

#         print()
#         printTop=10
#         for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
#             print("\tposs\t%s %s" % (v,k))
#         print()
#         for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
#             print("\tagent\t%s %s" % (v,k))        
#         print()
#         for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
#             print("\tpatient\t%s %s" % (v,k))       
#         print()
#         for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
#             print("\tmod\t%s %s" % (v,k))    
#         print()

In [ ]:
# import tarfile
# my_tar = tarfile.open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/MovieSummaries.tar.gz')
# my_tar.extractall('./CMU_Summaries')
# my_tar.close()

In [15]:
# with open('/content/drive/MyDrive/Visual Story Telling/Dataset - Story Generation/SampleBabylon.txt','r') as file:
#   data=file.read()
# 
# sentences = data.split('.')

In [5]:
import pandas as pd

In [ ]:
df_sentences = pd.read_table('/content/test_booknlp/sample_babylon.tokens')
# df_sentences['sentence'] = df_sentences.groupby(['sentence_ID'])['word'].apply(lambda x: ' '.join(x)).reset_index(drop=True)
# df_sentences['start'] = 
sentences = df_sentences.groupby(['sentence_ID'])['word'].apply(lambda x: ' '.join(x)).values

In [ ]:
# df_sentences['token_ID_within_document']

In [ ]:
# df_sentences['sentence'].dropna()

0      Alien envoys come to the giant space station i...
1      In the earth year 2257 , a multitude of humans...
2      Some of them are members of the station crew ,...
3      Others are ambassadors from major alien govern...
4      Still others are refugees , smugglers , busine...
                             ...                        
127    Ivanova explains why she has been avoiding Win...
128    The drugs lead Ivanova 's mother into a deep d...
129    Garibaldi has convinced Ambassador Delenn to c...
130    Sinclair watches election returns , showing a ...
131                                              < EOS >
Name: sentence, Length: 132, dtype: object

In [ ]:
# df_sentences.head(-5)

In [10]:
df = pd.read_table('/content/test_booknlp/sample_babylon.entities')

In [11]:
df_verb = pd.read_table('/content/test_booknlp/sample_babylon.supersense')

In [ ]:
# df_verb.head()

In [ ]:
# df.head()

In [47]:
text_values = df.text.values
ref_id = df.COREF.values 
cat = df.cat.values
start_token_id = df.start_token.values
type_of_identity = df.prop.values
extracted = []

In [26]:
sentences = sentences.tolist()

In [27]:
len(sentences)

132

In [42]:
lengths = [len(x.split()) for x in sentences]

In [ ]:
# len(sentences[0].split())

In [ ]:
# lenghts

In [44]:
for i in range(len(lenghts)):
  if i!=0:
    lengths[i] = lengths[i]+lengths[i-1]

In [ ]:
# lenghts

In [28]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [41]:
# df_sentences

,paragraph_ID,sentence_ID,token_ID_within_sentence,token_ID_within_document,word,lemma,byte_onset,byte_offset,POS_tag,fine_POS_tag,dependency_relation,syntactic_head_ID,event
0,0,0,0,0,Alien,alien,0,5,ADJ,JJ,amod,1,O
1,0,0,1,1,envoys,envoy,6,12,NOUN,NNS,nsubj,2,O
2,0,0,2,2,come,come,13,17,VERB,VBP,ROOT,2,EVENT
3,0,0,3,3,to,to,18,20,ADP,IN,prep,2,O
4,0,0,4,4,the,the,21,24,DET,DT,det,7,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,0,130,13,2946,Santiago,Santiago,15573,15581,PROPN,NNP,appos,2944,O
2947,0,130,14,2947,.,.,15581,15582,PUNCT,.,punct,2934,O
2948,0,131,0,2948,<,<,15584,15585,X,XX,nmod,2949,O
2949,0,131,1,2949,EOS,EOS,15585,15588,PROPN,NNP,ROOT,2949,O


In [40]:
# test = df_sentences.groupby(['sentence_ID'])['token_ID_within_document'].apply(lambda x: ' '.join(str(x))).values.tolist()

In [38]:
# print(test)

In [49]:
for j in range(len(sentences)):
  check = False
  identities = set()
  for i in range(len(text_values)):
    if start_token_id[i]<lengths[j-1] and i!=0:
      continue
    if start_token_id[i]>lengths[j]:
      break
    if text_values[i] in sentences[j] and cat[i]=='PER' and type_of_identity[i]=="PROP":
      identities.add((text_values[i], ref_id[i]))
  if len(identities)==2:
    check = True
  for idx, row in df_verb.iterrows():
    if "verb.social" in row.supersense_category and check:
      if row.start_token>lengths[j]:
        break  
      if row.start_token<lengths[j-1] and i!=0:
        continue
      print('###################################')
      print(f' \t Sentence {j}')
      print('###################################')
      print(identities)
      print(f"Verb: {row.text} and type: {row.supersense_category}")
      print(sentences[j])
      ss = SentimentIntensityAnalyzer().polarity_scores(sentences[j])
      del ss['compound']
      print(max(ss, key=ss.get), ss[max(ss, key=ss.get)])
      print()

###################################
 	 Sentence 15
###################################
{('Kosh', 28), ('Ambassador Londo Mollari', 26)}
Verb: failed and type: verb.social
Ambassador Londo Mollari quickly becomes a suspect in the case , as he had failed to show up at the appointed time with the other ambassadors to welcome Kosh aboard .
neu 0.714

###################################
 	 Sentence 18
###################################
{('Kosh', 28), ('Lyta', 29)}
Verb: persuade and type: verb.social
Takashima persuade Lyta to perform an unauthorized mind scan on the unconscious Kosh .
neu 1.0

###################################
 	 Sentence 20
###################################
{('Sinclair', 19), ('Lyta', 29)}
Verb: attempting and type: verb.social
Lyta accuses Sinclair of attempting to murder the ambassador .
neg 0.504

###################################
 	 Sentence 20
###################################
{('Sinclair', 19), ('Lyta', 29)}
Verb: murder and type: verb.social
Lyta accuses S

Now accumulate all the pairs and then apply sentiment analyzer

In [71]:
paired_sent = {}

In [72]:
for j in range(len(sentences)):
  check = False
  identities = []
  for i in range(len(text_values)):
    if start_token_id[i]<lengths[j-1] and i!=0:
      continue
    if start_token_id[i]>lengths[j]:
      break
    if text_values[i] in sentences[j] and cat[i]=='PER' and type_of_identity[i]=="PROP":
      identities.append([text_values[i], ref_id[i]])
  if len(identities)==2 and identities[1][0]!=identities[0][0]:
    # print(identities)
    check = True
  for idx, row in df_verb.iterrows():
    if "verb.social" in row.supersense_category and check:
      if row.start_token>lengths[j]:
        break  
      if row.start_token<lengths[j-1] and i!=0:
        continue
      # print('###################################')
      # print(f' \t Sentence {j}')
      # print('###################################')
      # print(identities)
      if f'{identities[0][0]} and {identities[1][0]}' in paired_sent or f'{identities[1][0]} and {identities[0][0]}' in paired_sent:
        if paired_sent.get(f'{identities[0][0]} and {identities[1][0]}')!=None:
          paired_sent[f'{identities[0][0]} and {identities[1][0]}'] += " "+sentences[j]
          break
        else:
          paired_sent[f'{identities[1][0]} and {identities[0][0]}'] += " "+sentences[j]
          break
      else:
        paired_sent[f'{identities[0][0]} and {identities[1][0]}'] = ""
        paired_sent[f'{identities[0][0]} and {identities[1][0]}'] += " "+sentences[j]
        break

      # print(f"Verb: {row.text} and type: {row.supersense_category}")
      # print(sentences[j])
      # ss = SentimentIntensityAnalyzer().polarity_scores(sentences[j])
      # del ss['compound']
      # print(max(ss, key=ss.get), ss[max(ss, key=ss.get)])
      # print()
      #Cannot use list for appending multiple sentences

In [73]:
paired_sent

{'Ambassador Londo Mollari and Kosh': ' Ambassador Londo Mollari quickly becomes a suspect in the case , as he had failed to show up at the appointed time with the other ambassadors to welcome Kosh aboard .',
 'Lyta and Kosh': ' Takashima persuade Lyta to perform an unauthorized mind scan on the unconscious Kosh .',
 'Lyta and Sinclair': ' Lyta accuses Sinclair of attempting to murder the ambassador .',
 "Ambassador G'Kar and Sinclair": " The Advisory Council considers a motion made by Ambassador G'Kar to turn Sinclair over to the Vorlon Empire , to stand trial for attempted murder .",
 'Ambassador Delenn and Ambassador Mollari': ' Takashima votes no ; Ambassador Delenn abstains ; Ambassador Mollari votes in favor .',
 'Sinclair and Mollari': ' Sinclair also tells Mollari he is calling Mollari Council and the League , hopeful he can form a coalition against the Narn attack .',
 'Sinclair and Garibaldi': ' After leaving the garden , Sinclair is met by Garibaldi , who has discovered that

In [78]:
relations = {'neu':"neutral",'pos':"positive",'neg':"negative"}
inter_char_relations = []
for key in paired_sent:
  ss = SentimentIntensityAnalyzer().polarity_scores(paired_sent[key])
  del ss['compound']
  # print(max(ss, key=ss.get), ss[max(ss, key=ss.get)])
  inter_char_relations.append(f'{key} have {relations[max(ss, key=ss.get)]} relationship')

In [79]:
inter_char_relations

['Ambassador Londo Mollari and Kosh have neutral relationship',
 'Lyta and Kosh have neutral relationship',
 'Lyta and Sinclair have negative relationship',
 "Ambassador G'Kar and Sinclair have neutral relationship",
 'Ambassador Delenn and Ambassador Mollari have neutral relationship',
 'Sinclair and Mollari have neutral relationship',
 'Sinclair and Garibaldi have neutral relationship',
 "Ambassador Delenn and G'Kar have neutral relationship",
 'Sinclair and the Raiders have neutral relationship',
 'Garibaldi and Mollari have neutral relationship',
 'Ivanova and Winters have neutral relationship']

In [58]:
# lol = {[[1,23],[2,12]]:45,[[3,676],[7,234]]:23}

TypeError: ignored

In [ ]:
# if "begins" in df_verb.text.values:
#   print(df_verb.supersense_category.values[list(df_verb.text.values).index("help")])

verb.social


In [ ]:
# type(df_verb.text.values)

numpy.ndarray